### Import modules

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import make_scorer
from sklearn.dummy import DummyRegressor

from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
# from catboost import CatBoostRegressor

import altair as alt


In [2]:
# Save a vega-lite spec and a PNG blob for each plot in the notebook
alt.renderers.enable('mimetype')
# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

## Read the data

In [3]:
# Read the data
df = pd.read_csv('data/PJME_hourly.csv')

# Remove spaces from column names
df.columns = df.columns.str.replace(' ', '')

# Convert Datetime column to datetime data type
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Set index to datetime column
df = df.set_index('Datetime')

# Sort the index to ascending order
df = df.sort_index()

# Rename the PJME_MW column to target column
df = df.rename(columns={'PJME_MW': 'target'})

# Check head of the resulting dataframe
df.head()

,target
Datetime,
2002-01-01 01:00:00,30393.0
2002-01-01 02:00:00,29265.0
2002-01-01 03:00:00,28357.0
2002-01-01 04:00:00,27899.0
2002-01-01 05:00:00,28057.0


## Split into train and test

In [4]:
df_train, df_test = train_test_split(df, test_size=0.2, shuffle=False, random_state=123)
print(df_train.shape)
print(df_test.shape)

(116292, 1)
(29074, 1)


## Visualize the data - Basic EDA

In [5]:
# Function to plot the raw data
def plot_data(data, x_axis, y_axis):
    """
    Plots line chart of the time series data (y-axis) against time (x-axis)

    Parameters
    ----------
    data: pd.DataFrame
        input dataframe
    x_axis: str
        the name of the time(datetime) column 
    y_axis: str
        the name of the time series data column

    Returns
    ----------
    lines: altair.vegalite.v4.api.Chart
        altair chart object 
    """
    data = data.reset_index()
    lines = alt.Chart(data).mark_line().encode(
        x=alt.X(x_axis),
        y=alt.Y(y_axis)
    ).properties(
        width=700,
        height=250
    )
    return lines

In [6]:
plot_data(df_train, x_axis='Datetime', y_axis='target')

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


## Separate train and test 

In [7]:
# Add lags
def generate_lags(df, num_lags):
    """
    Creates new features which are lags of the target variable
    Parameters
    ----------
    df: pd.DataFrame
        input dataframe
    num_lags: int
        the number of lags

    Returns
    ----------
    df_tmp: pd.DataFrame
        dataframe with lagged features created
    """
    df_tmp = df.copy()
    for n in range(1, num_lags+1):
        df_tmp[f'lag{n}'] = df_tmp['target'].shift(n)
    df_tmp = df_tmp.iloc[num_lags:]
    return df_tmp

In [8]:
tot_lags = 10
df_train = generate_lags(df=df_train, num_lags=tot_lags)
df_test = generate_lags(df=df_test, num_lags=tot_lags)

In [9]:
df_train.head()

,target,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,lag10
Datetime,,,,,,,,,,,
2002-01-01 11:00:00,31395.0,30692.0,29943.0,29595.0,29308.0,28654.0,28057.0,27899.0,28357.0,29265.0,30393.0
2002-01-01 12:00:00,31496.0,31395.0,30692.0,29943.0,29595.0,29308.0,28654.0,28057.0,27899.0,28357.0,29265.0
2002-01-01 13:00:00,31031.0,31496.0,31395.0,30692.0,29943.0,29595.0,29308.0,28654.0,28057.0,27899.0,28357.0
2002-01-01 14:00:00,30360.0,31031.0,31496.0,31395.0,30692.0,29943.0,29595.0,29308.0,28654.0,28057.0,27899.0
2002-01-01 15:00:00,29798.0,30360.0,31031.0,31496.0,31395.0,30692.0,29943.0,29595.0,29308.0,28654.0,28057.0


In [10]:
df_test.head()

,target,lag1,lag2,lag3,lag4,lag5,lag6,lag7,lag8,lag9,lag10
Datetime,,,,,,,,,,,
2015-04-10 01:00:00,26208.0,27846.0,30060.0,32363.0,33755.0,33843.0,33012.0,32707.0,32209.0,32049.0,32204.0
2015-04-10 02:00:00,25240.0,26208.0,27846.0,30060.0,32363.0,33755.0,33843.0,33012.0,32707.0,32209.0,32049.0
2015-04-10 03:00:00,24753.0,25240.0,26208.0,27846.0,30060.0,32363.0,33755.0,33843.0,33012.0,32707.0,32209.0
2015-04-10 04:00:00,24694.0,24753.0,25240.0,26208.0,27846.0,30060.0,32363.0,33755.0,33843.0,33012.0,32707.0
2015-04-10 05:00:00,25232.0,24694.0,24753.0,25240.0,26208.0,27846.0,30060.0,32363.0,33755.0,33843.0,33012.0


In [11]:
X_train, y_train = (
    df_train.drop(columns=['target']),
    df_train['target']
)
X_test, y_test = (
    df_test.drop(columns=['target']),
    df_test['target']
)

# Print shapes of the resulting train and test data sets
print(f'Shape of X_train and y_train: {X_train.shape, y_train.shape}')
print(f'Shape of X_test and y_test: {X_test.shape, y_test.shape}')

Shape of X_train and y_train: ((116282, 10), (116282,))
Shape of X_test and y_test: ((29064, 10), (29064,))


## Feature Engineering Using Pipeline Transformations

#### Custom Transformer for Automated Feature extraction

In [12]:
class UniveriateTimeSeriesAddFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, num_lags=10):
        self.num_lags = num_lags
        
    def __extract_features(self, X):
        """
        Creates new features from timestamp
        Parameters
        ----------
        X: pd.DataFrame
            input dataframe
        
        Returns
        ----------
        df_features: pd.DataFrame
            dataframe with new features created
        """
        df_features = X.copy()
        df_features = (
                df_features
                .assign(hour = df_features.index.hour)
                .assign(day = df_features.index.day)
                .assign(month = df_features.index.month)
                .assign(day_of_week = df_features.index.dayofweek)
                # .assign(week_of_year = df.index.isocalendar().week)
        )
        df_features = df_features.reset_index()
        return df_features
    
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df_features = X.copy()
        df_features = self.__extract_features(df_features)
        df_features = df_features.iloc[:,1:]
        return df_features

In [13]:
preprocessor = Pipeline(
    [
        ('add_lag_step', UniveriateTimeSeriesAddFeatures(num_lags=10)),
        ('stdscaler', StandardScaler())
    ]
)

In [ ]:
# preprocessor.fit_transform(X_train)

In [ ]:
# preprocessor.fit_transform(X_train).shape

#### Models

In [14]:
def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))


# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

In [15]:
# Function to collate results - Attribution to UBC MDS 571/573 course code for this function
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [16]:
results = {}

#### Dummy Regressor

In [17]:
dummy = DummyRegressor()

results["Dummy"] = mean_std_cross_val_scores(
    dummy, X_train, y_train, return_train_score=True, scoring=scoring_metrics
)

pd.DataFrame(results)

,Dummy
fit_time,0.008 (+/- 0.008)
score_time,0.001 (+/- 0.001)
test_neg RMSE,-6437.908 (+/- 249.131)
train_neg RMSE,-6433.014 (+/- 63.618)
test_r2,-0.007 (+/- 0.006)
train_r2,0.000 (+/- 0.000)
test_mape,-15.875 (+/- 0.611)
train_mape,-15.854 (+/- 0.091)


#### Other Models

In [18]:
pipe_ridge = make_pipeline(preprocessor, Ridge(random_state=123))
pipe_xgb = make_pipeline(preprocessor, XGBRegressor(random_state=123))
pipe_rf = make_pipeline(preprocessor, RandomForestRegressor(random_state=123))

models = {
    'ridge': pipe_ridge,
    'xgboost': pipe_xgb,
    'rf_regressor': pipe_rf
}

In [19]:
for (name, model) in models.items():
    results[name] = mean_std_cross_val_scores(
        model, X_train, y_train, return_train_score=True, scoring=scoring_metrics
    )

pd.DataFrame(results)

,Dummy,ridge,xgboost,rf_regressor
fit_time,0.008 (+/- 0.008),0.049 (+/- 0.022),4.862 (+/- 0.029),73.127 (+/- 0.349)
score_time,0.001 (+/- 0.001),0.008 (+/- 0.000),0.023 (+/- 0.005),0.752 (+/- 0.058)
test_neg RMSE,-6437.908 (+/- 249.131),-664.090 (+/- 30.887),-329.928 (+/- 17.854),-329.608 (+/- 22.237)
train_neg RMSE,-6433.014 (+/- 63.618),-664.094 (+/- 7.687),-279.970 (+/- 3.244),-118.130 (+/- 0.982)
test_r2,-0.007 (+/- 0.006),0.989 (+/- 0.001),0.997 (+/- 0.000),0.997 (+/- 0.001)
train_r2,0.000 (+/- 0.000),0.989 (+/- 0.000),0.998 (+/- 0.000),1.000 (+/- 0.000)
test_mape,-15.875 (+/- 0.611),-1.428 (+/- 0.053),-0.729 (+/- 0.046),-0.674 (+/- 0.057)
train_mape,-15.854 (+/- 0.091),-1.426 (+/- 0.016),-0.633 (+/- 0.008),-0.239 (+/- 0.003)


#### Best Model (Example of XGBRegressor provided below)

In [20]:
pipe_xgb.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('add_lag_step',
                                  UniveriateTimeSeriesAddFeatures()),
                                 ('stdscaler', StandardScaler())])),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='',
                              learning_rate=0.300000012, max_delta_step=0,
                              max_depth=6, min_child_weight=1, missing=nan,
                              monotone_constraints='()', n_estimators=100,
                              n_jobs=8, num_parallel_tree=1, random_state=123,
                              reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                              subsample=1, tree_me

In [21]:
preds = pipe_xgb.predict(X_test)

In [22]:
actual_vs_pred_df = y_test.reset_index()
actual_vs_pred_df['predicted'] = preds
actual_vs_pred_df.head()

,Datetime,target,predicted
0,2015-04-10 01:00:00,26208.0,25811.210938
1,2015-04-10 02:00:00,25240.0,24747.173828
2,2015-04-10 03:00:00,24753.0,24642.636719
3,2015-04-10 04:00:00,24694.0,24422.410156
4,2015-04-10 05:00:00,25232.0,25008.623047


In [23]:
actual_vs_pred_df_long = pd.melt(actual_vs_pred_df, id_vars='Datetime')

actual_vs_pred_df_long.tail()

,Datetime,variable,value
58123,2018-08-02 20:00:00,predicted,44161.867188
58124,2018-08-02 21:00:00,predicted,43458.199219
58125,2018-08-02 22:00:00,predicted,41937.488281
58126,2018-08-02 23:00:00,predicted,38635.917969
58127,2018-08-03 00:00:00,predicted,34957.195312


In [24]:
# Visualize the actual vs predicted chart for the last 2 days of the test data period
alt.Chart(actual_vs_pred_df_long[actual_vs_pred_df_long.Datetime > '2018-07-24']).mark_line().encode(
    x=alt.X('Datetime', type='temporal', title='Datetime'),
    y=alt.Y('value', type='quantitative', title='Actual vs Predicted'),
    color=alt.Color('variable', title='Type')
).properties(
    width=700
).interactive()

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [26]:
print(f'MAPE for test set: {mape(actual_vs_pred_df["target"], actual_vs_pred_df["predicted"])}')

MAPE for test set: 0.824815046658205
